In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
import tensorflow
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'lillian_modules'

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tensorflow.set_random_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)

#### The Data

In [ ]:
class_dict = {'No anemia': 0, 'Hemolytic anemia': 1, 'Aplastic anemia': 2, 'Iron deficiency anemia': 3,
              'Vitamin B12/Folate deficiency anemia': 4, 'Anemia of chronic disease': 5, 'Unspecified anemia': 6}

In [ ]:
#df = pd.read_csv('data/anemia_synth_dataset_hb_some_nans.csv') #my real dataset i think
df = pd.read_csv('data/anemia_synth_dataset_with_unspecified.csv')
#df = pd.read_csv('data/anemia_synth_dataset_hb.csv')
#df = pd.read_csv('data/noisy_dataset.csv')
df = df.fillna(0)
# classes = list(df.label.unique())
# nums = [i for i in range(len(classes))]
# class_dict = dict(zip(classes, nums))
# class_dict

In [ ]:
df.head()

In [ ]:
df['label'] = df['label'].replace(class_dict)
print(df.label.value_counts())
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
len(df)

In [ ]:
action_list = list(class_dict.keys()) + [col  for col in df.columns if col!='label']
len(action_list)

In [ ]:
action_list

In [ ]:
X_train[:2], X_test[:2]

#### The Environment

In [ ]:
from envs import SyntheticComplexHbEnv

#### The Agent

In [ ]:
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
from stable_baselines import DQN
from stable_baselines import bench, logger

In [ ]:
def stable_dqn():
    training_env = SyntheticComplexHbEnv(X_train, y_train)
    env = bench.Monitor(training_env, logger.get_dir())
    model = DQN('MlpPolicy', training_env, verbose=1, seed=SEED, n_cpu_tf_sess=1)
    model.learn(total_timesteps=int(1.5e6), log_interval=10000)
    #model.learn(total_timesteps=int(1.2e5), log_interval=10000)
    #model.save('models/synthetic_stable_dqn_1.8.pkl')
    #model.save('models/noisy/dqn_6e6.pkl')
    model.save('models/unspecified/dqn_15e6.pkl')
    env.close()
    return model

dqn_model = stable_dqn()

#### Performance Evaluation

In [ ]:
#dqn_model = DQN.load('models/synthentic_with_hb_some_nans_stable_dqn2e6.pkl')
dqn_model = DQN.load('models/unspecified/dqn_2e6.pkl')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, auc, roc_curve

In [ ]:
def multiclass(actual_class, pred_class, average = 'macro'):

    unique_class = set(actual_class)
    roc_auc_dict = {}
    for per_class in unique_class:
        other_class = [x for x in unique_class if x != per_class]
        new_actual_class = [0 if x in other_class else 1 for x in actual_class]
        new_pred_class = [0 if x in other_class else 1 for x in pred_class]
        roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
        roc_auc_dict[per_class] = roc_auc
    avg = sum(roc_auc_dict.values()) / len(roc_auc_dict)
    return avg

In [ ]:
def test(ytest, ypred):
    acc = accuracy_score(ytest, ypred)
    f1 = f1_score(ytest, ypred, average ='macro', labels=np.unique(ytest))
    try:
        roc_auc = multiclass(ytest, ypred)
    except:
        roc_auc = None
    return acc, f1, roc_auc

In [ ]:
def get_avg_length_reward(df):
    length = np.mean(df.episode_length)
    reward = np.mean(df.reward)
    return length, reward

In [ ]:
def synthetic_dqn_eval(dqn_model):
    test_df = pd.DataFrame()

    env = SyntheticComplexHbEnv(X_test, y_test, random=False)
    #env = SyntheticComplexHbEnv(X_train, y_train, random=False)
    count=0

    try:
        while True:
            count+=1
            if count%5000==0:
                print(f'Count: {count}')
            obs, done = env.reset(), False
            while not done:
                action, _states = dqn_model.predict(obs, deterministic=True)
                obs, rew, done,info = env.step(action)
                #if (done==True) & (np.isfinite(info['y_pred'])):
                if done == True:
                    test_df = test_df.append(info, ignore_index=True)
                #print('....................TEST DF ....................')
                #if len(test_df) != 0:
                #    print(test_df.head())

    except StopIteration:
        print('Testing done.....')
    return test_df

test_df = synthetic_dqn_eval(dqn_model)

In [ ]:
test_df.head()

In [ ]:
len(X_test), len(test_df)

In [ ]:
y_pred_df = test_df[test_df['y_pred'].notna()]
success_df = y_pred_df[y_pred_df['y_pred']== y_pred_df['y_actual']]
len(success_df)

In [ ]:
test_df = pd.read_csv('test_dfs/test_df_with_hb_some_nans_2e6.csv')
test_df.head(10)

In [ ]:
test_df.episode_length.mean()

In [ ]:
test_df.to_csv('test_dfs/unspecified/test_df_2e6.csv', index=False)
y_pred_df.to_csv('test_dfs/unspecified/y_pred_df_2e6.csv', index=False)
success_df.to_csv('test_dfs/unspecified/success_df_2e6.csv', index=False)

In [ ]:
y_pred_df.y_pred.unique()

In [ ]:
y_pred_df.y_pred.value_counts()

In [ ]:
y_pred_df.y_actual.value_counts()

In [ ]:
success_rate = len(success_df)/len(test_df)*100
success_rate

In [ ]:
#avg length and return 
avg_length, avg_return = get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = test(y_pred_df['y_actual'], y_pred_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df[test_df.y_actual==6].head()

#### Confusion Matrix and Classification Report

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
def plot_confusion_matrix(cm, save=False, filename=False):
    cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5], columns = [0, 1, 2, 3, 4, 5])
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_df, annot=True)
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Anemia')
    plt.xlabel('Predicted Anemia')
    plt.tight_layout()
    if save:
        plt.savefig(filename)
    plt.show()
    plt.close()

In [ ]:
#plot_confusion_matrix(confusion_matrix(y_pred_df['y_actual'], y_pred_df['y_pred']))

In [ ]:
def cm2inch(*tupl):
    inch = 2.54
    if type(tupl[0]) == tuple:
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)

In [ ]:
def show_values(pc, fmt="%.2f", **kw):    
    pc.update_scalarmappable()
    ax = pc.axes
    for p, color, value in zip(pc.get_paths(), pc.get_facecolors(), pc.get_array()):
        x, y = p.vertices[:-2, :].mean(0)
        if np.all(color[:3] > 0.5):
            color = (0.0, 0.0, 0.0)
        else:
            color = (1.0, 1.0, 1.0)
        ax.text(x, y, fmt % value, ha="center", va="center", color=color, **kw)

In [ ]:
def heatmap(AUC, title, xlabel, ylabel, xticklabels, yticklabels, figure_width=40, figure_height=20, correct_orientation=False, cmap='RdBu'):
    fig, ax = plt.subplots()    
    c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap=cmap)
    ax.set_yticks(np.arange(AUC.shape[0]) + 0.5, minor=False)
    ax.set_xticks(np.arange(AUC.shape[1]) + 0.5, minor=False)
    ax.set_xticklabels(xticklabels, minor=False)
    ax.set_yticklabels(yticklabels, minor=False)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)      

    # Remove last blank column
    plt.xlim( (0, AUC.shape[1]) )

    # Turn off all the ticks
    ax = plt.gca()    
    for t in ax.xaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False
    for t in ax.yaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False

    # Add color bar
    plt.colorbar(c)

    # Add text in each cell 
    show_values(c)

    # Proper orientation (origin at the top left instead of bottom left)
    if correct_orientation:
        ax.invert_yaxis()
        ax.xaxis.tick_top()       

    # resize 
    fig = plt.gcf()
    fig.set_size_inches(cm2inch(figure_width, figure_height))

In [ ]:
def plot_classification_report(classification_report, save=False, filename=False, cmap='RdBu'):
    lines = classification_report.split('\n')
    class_names = list(class_dict.keys())
    plotMat = []
    support = []
    #class_names = []
    #count = 0
    for line in lines[2 : (len(lines) - 5)]:
        t = line.strip().split()
        if len(t) < 2: continue
        v = [float(x) for x in t[1: len(t) - 1]]
        support.append(int(t[-1]))
        plotMat.append(v)

    xlabel = 'Metrics'
    ylabel = 'Classes'
    xticklabels = ['Precision', 'Recall', 'F1-score']
    ytick_labels = [f'{class_names[i]}({sup})' for i, sup in enumerate(support) ]
    
    #print(len(support))
    yticklabels = ['{0} ({1})'.format(class_names[idx], sup) for idx, sup  in enumerate(support)]
    figure_width = 25
    figure_height = len(class_names) + 7
    correct_orientation = False
    heatmap(np.array(plotMat), 'classification report', xlabel, ylabel, xticklabels, yticklabels, figure_width, figure_height, correct_orientation, cmap=cmap)
    #plt.tight_layout()
    if save:
        plt.savefig(filename, bbox_inches = 'tight')
    plt.show()
    plt.close()

In [ ]:
cr = classification_report(y_pred_df['y_actual'], y_pred_df['y_pred'])
plot_classification_report(cr)

In [ ]:
plot_classification_report(cr)